In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [15]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [16]:
# Updated Data Loading and Preprocessing
class SignatureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        self.labels = []
        
        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                self.images.append(os.path.join(class_path, img_name))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image, label

In [17]:
# Data Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
])

# Load and split the dataset
def load_and_split_data(root_dir, train_ratio=0.8):
    full_dataset = SignatureDataset(root_dir=root_dir, transform=transform)
    train_size = int(train_ratio * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    return train_dataset, test_dataset

In [18]:
# VAE Implementation (unchanged)
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.ReLU(),
            nn.Flatten()
        )
        
        self.fc_mu = nn.Linear(256 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(256 * 4 * 4, latent_dim)
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [19]:
# GAN Implementation (unchanged)
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [20]:
# Training functions (slightly modified)
def train_vae(vae, optimizer, train_loader, num_epochs=100):
    vae.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch, _ in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = vae(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

def train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader, num_epochs=100):
    criterion = nn.BCELoss()
    for epoch in range(num_epochs):
        for batch, _ in train_loader:
            batch = batch.to(device)
            batch_size = batch.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            outputs = discriminator(batch)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(batch_size, generator.latent_dim).to(device)
            fake_images = generator(noise)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()
            d_loss = d_loss_real + d_loss_fake
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [21]:
# Generating New Signatures (updated for group-based generation)
def generate_vae_signatures(vae, num_samples=10, group_idx=None):
    vae.eval()
    with torch.no_grad():
        z = torch.randn(num_samples, vae.latent_dim).to(device)
        if group_idx is not None:
            pass
        samples = vae.decode(z)
    return samples

def generate_gan_signatures(generator, num_samples=10, group_idx=None):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_samples, generator.latent_dim).to(device)
        if group_idx is not None:
            pass
        samples = generator(noise)
    return samples

# Metrics and Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch, _ in test_loader:
            batch = batch.to(device)
            recon_batch, mu, logvar = model(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [22]:
latent_dim = 100
root_dir = 'Grouped_Output'
    
    # Load and split the data
train_dataset, test_dataset = load_and_split_data(root_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # VAE training
vae = VAE(latent_dim).to(device)
vae_optimizer = optim.Adam(vae.parameters())
train_vae(vae, vae_optimizer, train_loader)
    

Epoch 1, Loss: 160422.1995
Epoch 2, Loss: 159347.2236
Epoch 3, Loss: 158751.6609
Epoch 4, Loss: 158059.5696
Epoch 5, Loss: 157183.5327
Epoch 6, Loss: 156689.3054
Epoch 7, Loss: 156461.6824
Epoch 8, Loss: 156131.3247
Epoch 9, Loss: 156011.7136
Epoch 10, Loss: 155585.1194
Epoch 11, Loss: 155100.2378
Epoch 12, Loss: 152109.0845
Epoch 13, Loss: 149654.5933
Epoch 14, Loss: 148794.0710
Epoch 15, Loss: 147444.4485
Epoch 16, Loss: 145857.8625
Epoch 17, Loss: 144927.4326
Epoch 18, Loss: 144248.4912
Epoch 19, Loss: 143396.3442
Epoch 20, Loss: 143534.5188
Epoch 21, Loss: 142582.5330
Epoch 22, Loss: 143233.6924
Epoch 23, Loss: 143123.6804
Epoch 24, Loss: 142641.7344
Epoch 25, Loss: 143147.5879
Epoch 26, Loss: 142453.2803
Epoch 27, Loss: 142372.2751
Epoch 28, Loss: 142048.8809
Epoch 29, Loss: 142579.9226
Epoch 30, Loss: 142576.4507
Epoch 31, Loss: 142346.5527
Epoch 32, Loss: 141941.6963
Epoch 33, Loss: 142172.1826
Epoch 34, Loss: 141283.3743
Epoch 35, Loss: 141989.0884
Epoch 36, Loss: 141204.7395
E

In [23]:
    # GAN training
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader)

Epoch [1/100], d_loss: 0.2056, g_loss: 3.4669
Epoch [2/100], d_loss: 0.0913, g_loss: 4.7284
Epoch [3/100], d_loss: 0.0592, g_loss: 5.1191
Epoch [4/100], d_loss: 0.0405, g_loss: 5.5071
Epoch [5/100], d_loss: 0.0310, g_loss: 5.8096
Epoch [6/100], d_loss: 0.0158, g_loss: 6.5883
Epoch [7/100], d_loss: 0.0286, g_loss: 7.3582
Epoch [8/100], d_loss: 2.4944, g_loss: 13.1855
Epoch [9/100], d_loss: 0.0994, g_loss: 4.1361
Epoch [10/100], d_loss: 0.0780, g_loss: 4.7233
Epoch [11/100], d_loss: 0.8668, g_loss: 7.9659
Epoch [12/100], d_loss: 0.4698, g_loss: 6.0076
Epoch [13/100], d_loss: 0.2992, g_loss: 3.2368
Epoch [14/100], d_loss: 0.1976, g_loss: 8.5597
Epoch [15/100], d_loss: 0.0780, g_loss: 5.0591
Epoch [16/100], d_loss: 0.0683, g_loss: 5.4908
Epoch [17/100], d_loss: 0.1305, g_loss: 4.6688
Epoch [18/100], d_loss: 0.0686, g_loss: 5.7530
Epoch [19/100], d_loss: 0.2793, g_loss: 9.5638
Epoch [20/100], d_loss: 0.0770, g_loss: 6.0474
Epoch [21/100], d_loss: 0.0182, g_loss: 5.3337
Epoch [22/100], d_los

In [25]:
    
    # Generate new signatures
vae_samples = generate_vae_signatures(vae)
gan_samples = generate_gan_signatures(generator)
    
    # Save generated samples
save_image(vae_samples, 'vae_generated_signatures.png')
save_image(gan_samples, 'gan_generated_signatures.png')
    
    # Evaluate models
vae_loss = evaluate_model(vae, test_loader)
print(f'VAE Test Loss: {vae_loss:.4f}')

# Generate group-specific signatures (placeholder)
num_groups = len(train_dataset.dataset.classes)
for group_idx in range(num_groups):
    group_name = train_dataset.dataset.classes[group_idx]
    vae_group_samples = generate_vae_signatures(vae, num_samples=5, group_idx=group_idx)
    gan_group_samples = generate_gan_signatures(generator, num_samples=5, group_idx=group_idx)
    save_image(vae_group_samples, f'vae_generated_signatures_group_{group_name}.png')
    save_image(gan_group_samples, f'gan_generated_signatures_group_{group_name}.png')

print("Training, evaluation, and group-specific generation complete!")

IndentationError: expected an indented block after 'for' statement on line 15 (3129987277.py, line 16)